In [2]:
import os
import sys
import copy
import time
import pickle
import numpy as np
from typing import *
import networkx as nx
from multiprocess import Process, Manager

from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag

import helper_functions as hf

# Surface Code Compilation and Resource Estimation #

This notebook will help the users compile a circuit to a quantum processor composed of surface code tiles. Each tile is connected to its nearest four neighbors (a grid topology - the vertices of the grid are the surface code qubits while the edges are the connections between qubits).

### Preprocess the circuit ###

For the purpose of this notebook, we assume that we are working with the wide surface code qubit as illusterated in https://quantum-journal.org/papers/q-2018-05-04-62/pdf/. The advantage of working with the wide surface code qubit is that it lets us perform all single qubit Clifford operations classically via a procedure called Edge tracking. This effectively eliminates the time overhead of all single qubit Clifford gates. Thus, we just need to worry about the CNOT gate (a two-qubit Clifford gate), and the T gate (non-Clifford gate). 

The following functions take our input circuit and convert them into a CNOT+T circuit. For this, we first decompose all single qubit Non-Clifford gates into Clifford+T using Gridsynth (https://www.mathstat.dal.ca/~selinger/newsynth/, https://arxiv.org/abs/1403.2975). Then, we get rid of all single qubit Clifford operations because of our edge-tracking assumption

In [3]:
def preprocessing(circuit):
    '''
    The circuit pre-processing steps
    '''

    # convert circuit into cx+u
    while circuit!=circuit.decompose():
        circuit=circuit.decompose()
    
    # convert into CNOT+T
    cliff_plus_T_circuit=hf.rewrite_circuit_in_cliff_plus_T(circuit)

    return cliff_plus_T_circuit

In [5]:
# change this to a two-step process where we just load a regular circuit and pre-process it ourselves
parent_folder='/Users/sid/Magic State Distillation/ArbitraryMagicStates/chris/scmr_input_circuits/'
input_circuit_name='adder_n28_Cliff+T.qasm'
cliff_T_circuit=QuantumCircuit.from_qasm_file(input_circuit_name)

### Initialize The Backend ###

Next, we will initialize the grid to which we would map our circuit. As mentioned before, the backend would be in the shape of a grid where vertices represent qubits and the edges represent connections between those qubits. We also need to define the locations of Magic State factories. These are the sources of Magic States which enable us to do Non-Clifford operations

In [6]:
def create_adjacency_matrix(V, E):
    '''
    Args:
    V: Number of vertices in the graph
    E: Edges in the graph

    Returns:
    Adjacency matrix of the device topology
    '''
    adj_matrix=[[0 for i in range(V)] for j in range(V)]
    for edge in E:
        adj_matrix[edge[0]][edge[1]]=1
        adj_matrix[edge[1]][edge[0]]=1
    
    return adj_matrix

print('Initializing the backend...')
dimension=100 # 100x100 grid --> 10000 qubits
G=nx.grid_2d_graph(dimension, dimension)
edge_list_temp=list(G.edges())
edge_list_rehashed=[(i[0]*dimension+i[1], j[0]*dimension+j[1]) for i, j in edge_list_temp]
all_ms=[5020, 5040, 5060, 5080] # Locations of the magic state factories. Each location is defined as a unique integer which is equal to dimension*x+y where x and y are the coordinates of the factory in the grid.
backend_graph=create_adjacency_matrix(dimension**2, edge_list_rehashed) # The adjacency matrix of the backend
print('Backend initialized!')

Initializing the backend...
Backend initialized!


### Mapping ###

The process of compilation can be broken into two steps - mapping and routing. During mapping, our aim is to assign logical surface code patches to qubits in the circuit. The mapping algorithm we use is the one given in Movali et. al. https://arxiv.org/pdf/2311.18042

##### Get Interaction Graph #####

As described in Molvai et. al., the first step in the mapping algorithm is to get the interaction graph of the circuit. The interaction graph captures the interaction betewen qubits (in the form of CNOTs) and between a qubit and a Magic state factory (in the form of a T gate). The following function can be used to obtain the interaction graph of the circuit

In [7]:
def get_interaction_graph(circuit:QuantumCircuit):
    '''
    Args:
    circuit: The input quantum circuit

    Gives an interaction graph as given in Molavi et. al. 2023 - Surface Code compilers
    '''
    circuit_dag=circuit_to_dag(circuit)
    
    # This interaction graph has a node 
    # for every qubit and every T gate
    interaction_graph={}
    num_qubits=circuit.num_qubits

    try:
        num_t_gates=circuit.count_ops()['t']
    except:
        num_t_gates=0

    # num t gates count
    num_t_gates_dict={}

    # Add nodes to the interaction graph
    for i in range(num_qubits):
        interaction_graph['q'+str(i)]=[]
        num_t_gates_dict['q'+str(i)]=0
    for i in range(num_t_gates):
        interaction_graph['t'+str(i)]=[]
    
    # Add edges to the interaction graph
    t_count=0 # counter keeping track of t-gates
    for node in circuit_dag.gate_nodes():
        if node.name=='cx':
            arg0='q'+str(node.qargs[0]._index)
            arg1='q'+str(node.qargs[1]._index)
            if arg0 not in interaction_graph[arg1]:
                interaction_graph[arg1].append(arg0)
            if arg1 not in interaction_graph[arg0]:
                interaction_graph[arg0].append(arg1)
        elif node.name=='t':
            interaction_graph['t'+str(t_count)].append('q'+str(node.qargs[0]._index))
            interaction_graph['q'+str(node.qargs[0]._index)].append('t'+str(t_count))
            num_t_gates_dict['q'+str(node.qargs[0]._index)]+=1
            t_count+=1
    
    return interaction_graph, num_t_gates_dict

In [30]:
list(circuit_to_dag(cliff_T_circuit).gate_nodes())[0].qargs[0]._index

1

In [8]:
interaction_graph, num_t_gates_dict=get_interaction_graph(cliff_T_circuit)

AttributeError: 'Qubit' object has no attribute 'index'

##### Get Interaction Chain Set #####

The interaction graph is broken into a set "chains" that can be mapped independently on to the backend. The chains are created in a greedy fashion from the interaction graph.

In [7]:
def single_t_dfs(interaction_graph:Dict[str, List[str]], vertex:str, visited:Dict[str, bool], path:List[str]):
    '''
    Args:
    interaction_graph: Interaction graph as a dictionary of lists. The index is a 
    vertex of the graph. The list contains the neighbors of the vertex (Adjacency list)
    vertex: The vertex at which we are currently at (during the dfs)
    visited: A dictionary keeping track of visited nodes in the interaction graph
    path: The current path that we are building during the dfs

    Returns:
    path: Updates the path and returns it
    
    '''

    def is_complete(path):
        '''Gives conditions for path to be complete -- i.e. we do
        not need to add any more vertices to it'''
        if len(path)>1 and path[-1][0]=='t':
            return True
        else:
            return False
    
    # if path is a complete path
    # just return it
    if is_complete(path):
        return path
    
    # assert that the path is one out of the two possible incomplete paths
    for el in path[1:]:
        assert el[0]=='q'
    
    # figure out whether a T gate is present in the path
    is_t_present=False
    for el in path:
        if el[0]=='t':
            is_t_present=True
            break
    
    if is_t_present and vertex[0]=='t':
        return path
    
    visited[vertex]=True
    path.append(vertex)

    # path is not complete yet -- keep going
    neighbors=interaction_graph[vertex]
    if vertex[0]=='q':
        for neighbor in neighbors:
            if not visited[neighbor]:
                path=single_t_dfs(interaction_graph, neighbor, visited, path)
            else:
                continue
    else:
        for neighbor in neighbors:
            if not visited[neighbor] and neighbor[0]=='q':
                path=single_t_dfs(interaction_graph, neighbor, visited, path)
            else:
                continue
    
    return path

def get_interaction_chain_set(interaction_graph:Dict[str, List[str]]):
    '''
    Args:
    interaction_graph: Interaction graph as a dictionary of lists. The index is a 
    vertex of the graph. The list contains the neighbors of the vertex (Adjacency list)

    Returns a set of interaction chains. It is a set of maximal paths such that 
    each vertex in the graph is in exactly one path. Further, each path must 
    contain at most one T gate
    '''
    # dictionary to keep track of visited nodes
    visited={node:False for node in interaction_graph.keys()}
    maximal_paths=[]
    all_keys=list(interaction_graph.keys())

    # dfs to find maximal paths
    while sum(visited.values())<len(visited.keys()):
        for vertex in all_keys:
            if not visited[vertex]:
                path=[]
                path=single_t_dfs(interaction_graph, vertex, visited, path) # dfs with the constraint that each path must contain at most one T gate -- that too either at the start or the end
                maximal_paths.append(path)
    
    return maximal_paths, visited

In [8]:
interaction_chain_set, visited=get_interaction_chain_set(interaction_graph)
assert sum(visited.values())==len(visited.keys()) # we need to visit all nodes in the interaction graph

##### Map the Independent Interaction Chains #####

In [15]:
def update_available_locations(available_locations:List[int], interaction_graph:Dict[str, List[str]], mapping:Dict[str, int], available_ms_factories_to_qubits:Dict[int, int]):
    '''
    Args:
    available_locations: List of available locations on the graph
    interaction_graph: Interaction graph of the circuit
    mapping: Mapping from virtual qubits to physical qubit locations in the graph in the form of a dictionary.
    available_ms_factories_to_qubits: Dictionary keeping track of the closest magic state factory to each mapped qubit
    '''
    updated_locations=[i for i in available_locations]
    invalid_nodes=all_ms+list(mapping.values())

    virtual_qubits=list(mapping.keys())
    
    if len(mapping.keys())>=2:
        for i in range(len(virtual_qubits)-1):
            for j in range(i, len(virtual_qubits)):

                physical_qbi=mapping[virtual_qubits[i]]
                physical_qbj=mapping[virtual_qubits[j]]

                # if these two qubits are connected in the circuit
                if virtual_qubits[i] in interaction_graph[virtual_qubits[j]]:
                    assert virtual_qubits[j] in interaction_graph[virtual_qubits[i]] # sanity check
                    shortest_path=hf.bfs_shortest_path(backend_graph, physical_qbi, physical_qbj, invalid_nodes=invalid_nodes)
                    assert shortest_path[0]==physical_qbi and shortest_path[-1]==physical_qbj # sanity check
                    for el_idx, el in enumerate(shortest_path):
                        
                        # make sure already mapped locations are not in available locations
                        if el_idx==0 or el_idx==len(shortest_path)-1:
                            assert el not in updated_locations
                        
                        if el in updated_locations:
                            updated_locations.remove(el)
                        
                    for el in shortest_path:
                        assert el not in updated_locations

    # make sure each of these mapped qubits have a way to reach at least one magic state factory        
    for i in range(len(virtual_qubits)):
        neighbors=interaction_graph[virtual_qubits[i]]
        flag=False
        for el in neighbors:
            if el[0]=='t':
                flag=True
                break

        if flag: # if this qubit interacts with a T gate
            shortest_length=np.inf
            shortest_route=None
            
            for ms_loc in all_ms:
                route=hf.bfs_shortest_path(backend_graph, ms_loc, mapping[virtual_qubits[i]], invalid_nodes=invalid_nodes)
                if route[-1]==mapping[virtual_qubits[i]] and route[0]==ms_loc: # valid route
                    
                    if len(route)<shortest_length:
                        shortest_length=len(route)
                        shortest_route=route
                        available_ms_factories_to_qubits[mapping[virtual_qubits[i]]]=ms_loc # record the closest magic state factory to mapped qubit
            
            assert shortest_route is not None # sanity check -- should have found at least one route

            for el in shortest_route[1:-1]:
                if el in updated_locations:
                    updated_locations.remove(el)

    return updated_locations, available_ms_factories_to_qubits

def map_chain(available_locations:List[int], interaction_chain:List[str], interaction_graph:Dict[str, List[str]], available_ms_factories_to_qubits:Dict[int, int]):
    '''
    Args:
    available_locations: List of available locations on the graph
    interaction_chain_set: Set of interaction chains
    interaction_graph: Interaction_graph of the circuit
    available_ms_factories_to_qubits: Dictionary keeping track of the closest magic state factory to each mapped qubit 
    '''
    graph=backend_graph
    ms=all_ms
    dimension=int(len(graph)**0.5)

    # dictionary to keep track of the mapping
    chain_mapping={}

    # make sure that 't' is always the last element in the chain
    if interaction_chain[0][0]=='t':
        interaction_chain.reverse()

    # base case when chain has just one element and its a T gate    
    if len(interaction_chain)==1 and interaction_chain[0][0]=='t':
        return chain_mapping, available_locations, available_ms_factories_to_qubits  
    # base case when the chain has just one element and its a qubit
    elif len(interaction_chain)==1 and interaction_chain[0][0]=='q':
        chain_mapping[interaction_chain[0]]=available_locations.pop()
        available_locations, available_ms_factories_to_qubits=update_available_locations(available_locations, interaction_graph, chain_mapping, available_ms_factories_to_qubits)
        return chain_mapping, available_locations, available_ms_factories_to_qubits
    else:
        head, p_dash = interaction_chain[0], interaction_chain[1:]
        new_chain_mapping, available_locations, available_ms_factories_to_qubits=map_chain(available_locations, p_dash, interaction_graph, available_ms_factories_to_qubits)
        chain_mapping.update(new_chain_mapping)

        # subset of available locations which are at 
        # distance 2 from the qubit mapping of p_dash[0]
        neighbors=[]
        
        # get all locations at distance two -- neighbors
        if len(p_dash)==1 and p_dash[0][0]=='t':
            for ms_loc in ms:
                co_ordinates=(ms_loc//dimension, ms_loc%dimension)
                potential_neighbors=[(co_ordinates[0]+i, co_ordinates[1]+j) for i in [-2, -1, 0, 1, 2] for j in [-2, -1, 0, 1, 2] if abs(i)+abs(j)==2]
                valid_neighbors=[i*dimension+j for i, j in potential_neighbors if i>=0 and i<dimension and j>=0 and j<dimension and i*dimension+j in available_locations]
                neighbors+=valid_neighbors
        else:
            head_p_dash_mapped=chain_mapping[p_dash[0]]
            co_ordinates=(head_p_dash_mapped//dimension, head_p_dash_mapped%dimension)
            potential_neighbors=[(co_ordinates[0]+i, co_ordinates[1]+j) for i in [-2, -1, 0, 1, 2] for j in [-2, -1, 0, 1, 2] if abs(i)+abs(j)==2]
            valid_neighbors=[i*dimension+j for i, j in potential_neighbors if i>=0 and i<dimension and j>=0 and j<dimension and i*dimension+j in available_locations]
            neighbors+=valid_neighbors
        
        if len(neighbors)!= 0:
            chain_mapping[head]=neighbors.pop()
            available_locations.remove(chain_mapping[head])
            available_locations, available_ms_factories_to_qubits=update_available_locations(available_locations, interaction_graph, chain_mapping, available_ms_factories_to_qubits)
        else:
            chain_mapping[head]=available_locations.pop()
            available_locations, available_ms_factories_to_qubits=update_available_locations(available_locations, interaction_graph, chain_mapping, available_ms_factories_to_qubits)
        
        return chain_mapping, available_locations, available_ms_factories_to_qubits

def structural_mapping_algorithm(circuit:QuantumCircuit):
    '''
    Args:
    circuit: The circuit that we want to map
    '''
    graph=backend_graph # the graph of the device connectivity
    ms=all_ms # magic state locations
    dimension=int(len(graph)**0.5) # dimension of the grid

    # get the interaction chain set
    interaction_graph, num_t_gates_dict=get_interaction_graph(circuit)
    interaction_chain_set, visited=get_interaction_chain_set(interaction_graph)

    # store available_locations in reverse order of
    # distance from the magic state factories -- the ones close to 
    # any magic state factory is preferred
    available_locations={}
    for i in range(len(graph)):
        if i in ms:
            continue
        else:
            co_ord=(i//dimension, i%dimension)
            dists=[abs(co_ord[0]-ms_loc//dimension)+abs(co_ord[1]-ms_loc%dimension) for ms_loc in ms]
            max_dist=max(dists)
            available_locations[i]=max_dist
    
    # sort the available locations according to value
    available_locations=sorted(available_locations, key=lambda x: available_locations[x], reverse=True)

    # run the map-chain algorithm
    final_mapping={}
    available_ms_factories_to_qubits={} # keeps track of the closest magic state factory to each mapped qubit
    for chain_idx, chain in enumerate(interaction_chain_set):
        mapping, available_locations, available_ms_factories_to_qubits=map_chain(available_locations, chain, interaction_graph, available_ms_factories_to_qubits)
        final_mapping.update(mapping)

    # refine final_mapping
    final_mapping_refined={int(key[1:]):val for key, val in final_mapping.items()}
    del final_mapping

    return final_mapping_refined, available_ms_factories_to_qubits

In [16]:
mapping, available_ms_factories_to_qubits=structural_mapping_algorithm(cliff_T_circuit)

/var/folders/w9/82hl73yj3_5937ld729s8j980000gp/T/ipykernel_18000/3162911751.py:34: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  arg0='q'+str(node.qargs[0].index)
/var/folders/w9/82hl73yj3_5937ld729s8j980000gp/T/ipykernel_18000/3162911751.py:35: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  arg1='q'+str(node.qargs[1].index)
/var/folders/w9/82hl73yj3_5937ld729s8j980000gp/T/ipykernel_18000/3

### Routing ###

We have obtained the mapping now. We know the co-ordinates of the logical qubit patches that the circuit qubits would be mapped to. The next step is to determine how the CNOTs and T gates would be executed. Both CNOTs and T gate execution require lattice surgery which involves initializing an ancilla patch between the control and target qubits. For any two ancilla patches, their ancilla patches cannot intersect (i.e. there cannot be a qubit which is common to multiple ancilla patches in the same time step). We also want to minimize the length of these ancilla patches so that the chances of crossover between two ancilla patches is minimized. Hence the problem of routing essentially translates to that of finding the set of shortest non-intersecting ancilla patches for all operations occuring in the same time step. 

If for any given operation, we cannot find an ancilla patch that doesn't intersect with all other ancilla patches in the same time step, we perform the operation in the next time step.

In [23]:
def clean_up_routing_output(routing_output):
    '''The routing output is stored in a really bad format -- clean that up'''
    
    sorted_output=sorted(routing_output, key=lambda x: x[0])
    
    # get the length of the routing list
    length=0
    for el in sorted_output:
        if len(el[1])>0:
            length+=len(el[1])

    clean_output={idx:None for idx in range(length)}

    counter=0
    for el_idx, el in enumerate(sorted_output):
        ll=el[1]
        if len(ll)>0:
            for element in ll:
                gate_routes={}
                for key, val in element.items():
                    assert key not in gate_routes
                    gate_routes[key]=val
                clean_output[counter]=gate_routes
                counter+=1

    assert counter==length      
    return clean_output

def shortest_find_ndp_algorithm(graph, pairs:Dict[int, Dict[str, Union[str, List[int]]]], mapping:Dict[int, int]):
    '''
    Args:
    pairs: Vertex pairs as a dictionary of dictionaries. Each dictionary contains  
    the vertices associated with the gate, and the name of the gate. The pairs dict is 
    queried using a unique_id.

    graph: Graph as an adjacency matrix of dimension VxV - numpy array
    pairs: Dictionary of pairs of vertices to be routed
    mapping: Mapping from virtual qubits to physical qubit locations in the graph in the form of a dictionary.

    Returns:
    All possible routes for that particular step
    '''
    relevant_pairs=copy.deepcopy(pairs)
    step={}
    
    while len(list(relevant_pairs.keys()))>0:

        # index corresponding to the closest vertices
        shortest_pidx=np.inf
        shortest_path=None
        shortest_length=np.inf
        shortest_unique_id=None

        # append the shortest path in step
        is_at_least_one_path=False 
        for unique_id, gate in relevant_pairs.items():
            
            # BFS does terminate, it finds something everytime
            assert (gate['indices'][0] != gate['indices'][1])
            path=hf.bfs_shortest_path(graph, gate['indices'][0], gate['indices'][1], all_ms+list(mapping.values()))

            try:
                assert path[0]==gate['indices'][0] and path[-1]==gate['indices'][1]
            except:
                continue
            
            # make sure that the bfs path is a valid path
            if path[0]==gate['indices'][0] and path[-1]==gate['indices'][1]:
                is_at_least_one_path=True

            if len(path)<shortest_length:
                shortest_path=path
                shortest_length=len(path)
                shortest_unique_id=unique_id
        
        # if no valid paths found, break the loop
        if not is_at_least_one_path:
            break
        
        step[shortest_unique_id]=shortest_path
        del relevant_pairs[shortest_unique_id]

        # remove all vertices in path from the graph
        for i in range(len(shortest_path)):
            graph[shortest_path[i], :]=0
            graph[:, shortest_path[i]]=0
    
    return step

def greedy_routing_algorithm(layer_infos, mapping:Dict[int, int], available_ms_factories_to_qubits:Dict[int, int], queue):
    '''
    Args:
    layer_infos: A list of dicts({'layer_idx':int, 'layer':DAGCircuitLayer}) containing information about the layers in the circuit
    mapping: Mapping from virtual qubits to physical qubit locations in the graph in the form of a dictionary.
    Assumes that keys are the virtual qubit indices and the values are the physical qubit indices.
    queue: Multiprocessing queue to store the results of the routing algorithm
    available_ms_factories_to_qubits: Dictionary keeping track of the closest magic state factory to each mapped qubit
    '''
    graph=np.array(backend_graph)
    ms=all_ms

    for layer_info in layer_infos:
        steps=[]
        layer_idx=layer_info['layer_idx']
        print('Current layer index is: ', layer_idx)
        layer=layer_info['layer']
        layer_graph=layer['graph']
        gate_nodes=layer_graph.gate_nodes()

        # get the set of all unrouted nodes
        unrouted={}
        unique_id=0 # Assign each gate in the layer with a unique id
        for gate_idx, gate in enumerate(gate_nodes):

            # The case of CNOT gates
            if len(gate.qargs)==2 and gate.name=='cx':
                post_mapping_indices=[mapping[gate.qargs[0].index], mapping[gate.qargs[1].index]]
                gate_relevant_vals={'name':gate.name, 'indices':post_mapping_indices}
                unrouted[unique_id]=gate_relevant_vals
            
            elif gate.name=='t' and len(gate.qargs)==1:
                
                # route from the closest magic state factory
                post_mapping_indices=[available_ms_factories_to_qubits[mapping[gate.qargs[0].index]], mapping[gate.qargs[0].index]] # defaulting to choosing the first magic state factory right now
                gate_relevant_vals={'name':gate.name, 'indices':post_mapping_indices}
                unrouted[unique_id]=gate_relevant_vals
            
            unique_id+=1
        
        # greedily route the gates
        t0=time.time()
        while len(list(unrouted.keys()))>0:

            graph_copy=graph.copy()
            next=shortest_find_ndp_algorithm(graph_copy, unrouted, mapping)
            
            steps.append(next)
            
            new_unrouted={}
            for el_idx, el_val in unrouted.items():
                if el_idx not in next:
                    new_unrouted[el_idx]=el_val
            
            unrouted=new_unrouted
        t1=time.time()
        queue.put([layer_idx, steps])
        print('Layer idx:', layer_idx, 'routing completed in time:', t1-t0, 'seconds!\n')

def parallel_greedy_routing_algorithm(circuit:QuantumCircuit, mapping:Dict[int, int], available_ms_factories_to_qubits:Dict[int, int]):
    '''
    Args:
    circuit: Quantum circuit to be routed as a DAG
    mapping: Mapping from virtual qubits to physical qubit locations in the graph in the form of a dictionary.
    Assumes that keys are the virtual qubit indices and the values are the physical qubit indices.
    available_ms_factories_to_qubits: Dictionary keeping track of the closest magic state factory to each mapped qubit
    '''
    def get_layers(circuit:QuantumCircuit):
        
        dag_circuit=circuit_to_dag(circuit)
        layers=list(dag_circuit.layers())
        return layers

    mapping_vals=list(mapping.values())
    available_qubit_locs_keys=list(available_ms_factories_to_qubits.keys())
    # for idx in mapping_vals:
    #     assert idx in available_qubit_locs_keys
    for idx in available_qubit_locs_keys:
        assert idx in mapping_vals
    # assert len(available_qubit_locs_keys)==len(mapping_vals)

    graph=np.array(backend_graph)
    ms=all_ms

    steps=[]
    layers=get_layers(circuit)
    layer_infos=[{'layer_idx':layer_idx, 'layer':layer} for layer_idx, layer in enumerate(layers)]

    print('Number of layers:', len(layers))

    def process_layer(layer_infos, result_queue):
        return greedy_routing_algorithm(layer_infos, mapping, available_ms_factories_to_qubits, result_queue)
    
    # fire all parallel processes
    result_queue=Manager().Queue()
    n_proc=os.cpu_count()-1
    len_each_proc=len(layer_infos)//n_proc
    processes=[]
    results=[]

    print('Launching processes!')
    for i in range(n_proc):
        print('Launched process number ', i, '!')
        process=Process(target=process_layer, args=(layer_infos[i*len_each_proc:(i+1)*len_each_proc], result_queue))
        processes.append(process)
        process.start()
    
    # wait for all processes to finish
    # and collect the results
    for process in processes:
        process.join()
    
    while not result_queue.empty():
        results.append(result_queue.get())
    
    results=clean_up_routing_output(results)
    return results

In [24]:
steps=parallel_greedy_routing_algorithm(cliff_T_circuit, mapping, available_ms_factories_to_qubits)

Number of layers: 306
Launching processes!
Launched process number  0 !
Launched process number  1 !
Launched process number  2 !
Launched process number  3 !
Launched process number  4 !
Launched process number  5 !
Launched process number  6 !
Current layer index is:  258Current layer index is: 
 129Current layer index is: 
Layer idx:  

/var/folders/w9/82hl73yj3_5937ld729s8j980000gp/T/ipykernel_18000/548014944.py:124: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  post_mapping_indices=[available_ms_factories_to_qubits[mapping[gate.qargs[0].index]], mapping[gate.qargs[0].index]] # defaulting to choosing the first magic state factory right now


258

/var/folders/w9/82hl73yj3_5937ld729s8j980000gp/T/ipykernel_18000/548014944.py:117: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  post_mapping_indices=[mapping[gate.qargs[0].index], mapping[gate.qargs[1].index]]


0
 Current layer index is: routing completed in time:  Layer idx:2158.821487426757812e-06 
 

...

Layer idx: 1 routing completed in time: 3.5988757610321045 seconds!

Current layer index is:  2
Layer idx: 87 routing completed in time: 1.737562656402588 seconds!

Current layer index is:  88


/var/folders/w9/82hl73yj3_5937ld729s8j980000gp/T/ipykernel_18000/548014944.py:117: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  post_mapping_indices=[mapping[gate.qargs[0].index], mapping[gate.qargs[1].index]]


Layer idx: 173 routing completed in time: 2.3288838863372803 seconds!

Current layer index is:  174
Layer idx: 46 routing completed in time: 1.8996152877807617 seconds!

Current layer index is:  47
Layer idx: 260 routing completed in time: 2.5933001041412354 seconds!

Current layer index is:  261
Layer idx: 216 routing completed in time: 2.8648109436035156 seconds!

Current layer index is: Layer idx:  217
174 routing completed in time: 1.5299923419952393 seconds!

Current layer index is:  175
Layer idx: 47 routing completed in time: 1.5228328704833984 seconds!

Current layer index is:  48
Layer idx: 261 routing completed in time: 1.9593451023101807 seconds!

Current layer index is: Layer idx:  26248
 routing completed in time: 0.49036693572998047 seconds!

Current layer index is:  49
Layer idx: 49 routing completed in time: 0.5663719177246094 seconds!

Current layer index is:  50
Layer idx: 50 routing completed in time: 5.0067901611328125e-06 seconds!

Current layer index is:  51
Layer

/var/folders/w9/82hl73yj3_5937ld729s8j980000gp/T/ipykernel_18000/548014944.py:124: DeprecationWarning: The property ``qiskit.circuit.bit.Bit.index`` is deprecated as of qiskit-terra 0.17. It will be removed no earlier than 3 months after the release date. Instead, use :meth:`~qiskit.circuit.quantumcircuit.QuantumCircuit.find_bit` to find all the containing registers within a circuit and the index of the bit within the circuit.
  post_mapping_indices=[available_ms_factories_to_qubits[mapping[gate.qargs[0].index]], mapping[gate.qargs[0].index]] # defaulting to choosing the first magic state factory right now


Layer idx: 270 routing completed in time: 0.7045578956604004 seconds!

Current layer index is:  271
Layer idx: 9 routing completed in time: 0.397611141204834 seconds!

Current layer index is:  10
Layer idx: 217 routing completed in time: 5.876224040985107 seconds!

Current layer index is:  218
Layer idx: 271 routing completed in time: 0.7818670272827148 seconds!

Current layer index is:  272
Layer idx: 177 routing completed in time: 1.9535999298095703 seconds!

Current layer index is:  178
Layer idx: 272 routing completed in time: 0.4819049835205078 seconds!

Current layer index is:  273
Layer idx: 273 routing completed in time: 0.32487988471984863 seconds!

Current layer index is:  274
Layer idx: 274 routing completed in time: 4.0531158447265625e-06 seconds!

Current layer index is:  275
Layer idx: 178 routing completed in time: 0.8477528095245361 seconds!

Current layer index is:  179
Layer idx: 275 routing completed in time: 0.8484318256378174 seconds!

Current layer index is:  276
